# DATA SPLIT - testing the full approach
saving models in a different folder using pickle, saving them in the dataframe causes xgboost to crash

**DO NOT RUN AGAIN**

In [15]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time
import uuid

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']
cognipath=datapath+"/COGNITION"
dreamspath=datapath+"/DREAMS"
masspath=datapath+"/MASS"


## define a fixed samplerate

In [16]:
samplerate=200

## load mass

In [17]:
#annotations, signalsMetadata = loadMASSSpindles(masspath,forceSamplerate=samplerate,onlySpindlesFilteredN2=True)
annotations, signalsMetadata = loadMASSSpindles(masspath,forceSamplerate=samplerate)

In [18]:
#check with EDA results (not needed)
print(len(annotations)/33458)

1.3461653416223325


In [19]:
annotations=annotations[annotations.type=='spindle'].reset_index(drop=True)

In [20]:
annotations.head()

,type,expert,subjectId,labelerId,startTime,duration,samplerate,stopTime,startInd,stopInd
0,spindle,E1,0001,0001,888.327805,0.640579,200,888.968384,177666,177794
1,spindle,E1,0001,0001,905.758061,0.578094,200,906.336155,181152,181267
2,spindle,E1,0001,0001,917.731574,0.847603,200,918.579177,183546,183716
3,spindle,E1,0001,0001,922.078189,0.878845,200,922.957034,184416,184591
4,spindle,E1,0001,0001,939.055445,0.757767,200,939.813212,187811,187963


In [21]:
signalsMetadata.head()

,subjectId,file,channel,duration,samplerate,isOriginalSamplerate,database
0,0001,MASS_0001.pkl,C3-CLE,28956.0,200,False,MASS
1,0002,MASS_0002.pkl,C3-CLE,35016.0,200,False,MASS
2,0003,MASS_0003.pkl,C3-CLE,36760.0,200,False,MASS
3,0004,MASS_0004.pkl,C3-CLE,28004.0,200,False,MASS
4,0005,MASS_0005.pkl,C3-CLE,31244.0,200,False,MASS


## create data split
5 fold CV, validating over training data as YOU et al 2021

In [22]:
#define subjects used in the experiment, keep only annotated by two experts
usedSubjects=np.array(signalsMetadata['subjectId'])
usedSubjects=np.setdiff1d(usedSubjects,['0004','0008','0015','0016'])
print(len(usedSubjects))
usedSubjects

15


array(['0001', '0002', '0003', '0005', '0006', '0007', '0009', '0010',
       '0011', '0012', '0013', '0014', '0017', '0018', '0019'],
      dtype=object)

In [23]:
testSplits=[]
noSelected=usedSubjects.copy()
for i in range(5):
    thisTestSplit=np.random.choice(noSelected,size=3,replace=False)
    testSplits.append(thisTestSplit)
    noSelected=np.setdiff1d(noSelected,thisTestSplit)


In [24]:
trainSplits=[]
valSplits=[]
for testSplit in testSplits:
    noTestSplit=np.setdiff1d(usedSubjects,testSplit)
    valSplit=np.random.choice(noTestSplit,3,replace=False)
    trainSplit=np.setdiff1d(noTestSplit,valSplit)
    trainSplits.append(trainSplit)
    valSplits.append(valSplit)

In [25]:
dataSplits=pd.DataFrame({
    'train':trainSplits,
    'val':valSplits,
    'test':testSplits
})

dataSplits

,train,val,test
0,"[0001, 0005, 0006, 0009, 0010, 0011, 0012, 001...","[0018, 0017, 0007]","[0013, 0002, 0003]"
1,"[0002, 0003, 0006, 0007, 0009, 0013, 0017, 001...","[0011, 0001, 0010]","[0005, 0014, 0012]"
2,"[0003, 0005, 0007, 0009, 0010, 0012, 0013, 001...","[0001, 0002, 0019]","[0011, 0006, 0018]"
3,"[0002, 0003, 0005, 0006, 0007, 0012, 0013, 001...","[0009, 0011, 0017]","[0019, 0001, 0010]"
4,"[0001, 0003, 0005, 0006, 0010, 0011, 0013, 001...","[0014, 0012, 0002]","[0009, 0017, 0007]"


In [26]:
dumpPickle("dataSplits_5Fold.pkl",dataSplits)